# Granite Route and Solve Agent

In this recipe, you will use the IBM [Granite](https://www.ibm.com/granite) model now available on watsonx.ai to build a Route and Solve Agent. This agent extends the [Function Calling Agent](../Function_Calling/Function_Calling_Agent.ipynb) recipe by introducing a router node that intelligently distributes queries to specialized subagents, each with their own grouped set of tools.

The Route and Solve architecture is ideal when you have a large number of tools that can be naturally grouped by category or domain. Instead of presenting all tools to a single agent, the router first determines which category of tools are needed, then routes the query to the appropriate subagent.

This approach offers several benefits:
- **Reduced tool selection errors**: Subagents only see relevant tools for their domain
- **Better scalability**: Easy to add new tool categories as subagents
- **Improved reasoning**: Specialized subagents can reason more effectively within their domain
- **Clear separation of concerns**: Tools are logically grouped by functionality


## Key Concepts in Route and Solve Architecture

### 1. **Router Node**
The router analyzes each query and determines which category of tools is most relevant. This reduces the likelihood of incorrect tool selection by limiting the agent's choices to a relevant subset.

### 2. **Subagents**
Each subagent is a specialized function calling agent that only has access to tools in its category. In this example:
- **Finance Subagent**: Has access to stock price and stock info tools
- **Weather Subagent**: Has access to current weather and forecast tools

### 3. **Looping Mechanism**
Within each subagent, we maintain the same loop structure as in the Function Calling Agent:
- LLM determines if a tool call is needed
- If yes, execute the tool and return to the LLM
- If no, the subagent completes and returns to the end node

### 4. **Scalability**
Adding new tool categories is straightforward:
1. Define new tools in a new category
2. Create a new subagent (LLM node and tool node)
3. Add the subagent nodes to the graph
4. Update the router to handle the new category
5. Add appropriate edges for the new subagent

## Agent Flow Diagram

```mermaid
graph TD
    Start([User Query])
    Router[Router Node]
    Finance[Finance Subagent]
    Weather[Weather Subagent]
    End[End Node]
    Response([Final Response])
    
    Start --> Router
    Router -->|finance query| Finance
    Router -->|weather query| Weather
    Router -->|other query| End
    Finance --> End
    Weather --> End
    End --> Response
```

This diagram shows how queries flow through the Route and Solve agent:
1. A user query enters the system
2. The **Router Node** analyzes it and determines the appropriate subagent category
3. The query is routed to the specialized **Finance Subagent** or **Weather Subagent**, or directly to the **End Node** for non-tool queries
4. The subagent processes the query and may execute tools as needed
5. The final response is returned to the user

# Steps

## Step 1. Set up your environment

While you can choose from several tools, this recipe is best suited for a Jupyter Notebook. Jupyter Notebooks are widely used within data science to combine code with various data sources such as text, images and data visualizations.

You can run this notebook in [Colab](https://colab.research.google.com/), or download it to your system and [run the notebook locally](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Getting_Started_with_Jupyter_Locally/Getting_Started_with_Jupyter_Locally.md).

To avoid Python package dependency conflicts, we recommend setting up a [virtual environment](https://docs.python.org/3/library/venv.html).

Note, this notebook is compatible with Python 3.12 and well as Python 3.11, the default in Colab at the time of publishing this recipe. To check your python version, you can run the `!python --version` command in a code cell.

## Step 2. Set up a watsonx.ai instance

See [Getting Started with IBM watsonx](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Getting_Started/Getting_Started_with_WatsonX.ipynb) for information on getting ready to use watsonx.ai.

You will need three credentials from the watsonx.ai set up to add to your environment: `WATSONX_URL`, `WATSONX_APIKEY`, and `WATSONX_PROJECT_ID`.

## Step 3. Install relevant libraries and set up credentials and the Granite model

We'll need a few libraries for this recipe. We will be using LangGraph and LangChain libraries to use Granite on watsonx.ai.

In [ ]:
! echo "::group::Install Dependencies"
%pip install uv
! uv pip install "git+https://github.com/ibm-granite-community/utils.git" \
    langgraph \
    langchain \
    langchain_ibm
! echo "::endgroup::"

Now we will get the credentials to use watsonx.ai and create the Granite model for use.

In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_core.utils.utils import convert_to_secret_str
from langchain.chat_models import init_chat_model

model = "ibm/granite-4-h-small"

llm_params = {
    "temperature": 0,
    "max_completion_tokens": 200,
    "repetition_penalty": 1.05,
}

# --- 1. LLM Initialization (Agent's Brain) ---
llm = init_chat_model(
    model=model,
    model_provider="ibm",
    url=convert_to_secret_str(get_env_var("WATSONX_URL")),
    apikey=convert_to_secret_str(get_env_var("WATSONX_APIKEY")),
    project_id=get_env_var("WATSONX_PROJECT_ID"),
    params=llm_params,
)
print(f"LLM initialized: {model}")



## Step 4: Define the functions grouped by category

In a Route and Solve agent, tools are organized into logical groups. Each group will be handled by a dedicated subagent. This organization allows the router to determine which category of tools is needed and route the query accordingly.

We'll create two categories: **Finance Tools** and **Weather Tools**. In a real-world scenario, you might have more categories such as Database Tools, Email Tools, or API Integration Tools.

These tools are sourced from the [Function Calling Agent](../Function_Calling/Function_Calling_Agent.ipynb) recipe to ensure consistency across recipes and enable maximum code reuse.

### Finance Tools

In [ ]:
import requests
from langchain_core.utils.utils import convert_to_secret_str

AV_STOCK_API_KEY = convert_to_secret_str(get_env_var("AV_STOCK_API_KEY", "unset"))

def get_stock_price(ticker: str, date: str) -> dict:
    """
    Retrieves the lowest and highest stock prices for a given ticker and date.

    Args:
        ticker: The stock ticker symbol, for example, "IBM".
        date: The date in "YYYY-MM-DD" format for which you want to get stock prices.

    Returns:
        A dictionary containing the low and high stock prices on the given date.
    """
    print(f"Getting stock price for {ticker} on {date}")

    apikey = AV_STOCK_API_KEY.get_secret_value()
    if apikey == "unset":
        print("No API key present; using a fixed, predetermined value for demonstration purposes")
        return {
            "low": "245.4500",
            "high": "249.0300"
        }

    try:
        stock_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={apikey}"
        stock_data = requests.get(stock_url)
        data = stock_data.json()
        stock_low = data["Time Series (Daily)"][date]["3. low"]
        stock_high = data["Time Series (Daily)"][date]["2. high"]
        return {
            "low": stock_low,
            "high": stock_high
        }
    except Exception as e:
        print(f"Error fetching stock data: {e}")
        return {
            "low": "none",
            "high": "none"
        }

print("=== Finance Tools Initialized ===")
print("  Tool 1: 'get_stock_price'")

### Weather Tools

In [ ]:
WEATHER_API_KEY = convert_to_secret_str(get_env_var("WEATHER_API_KEY", "unset"))

def get_current_weather(location: str) -> dict:
    """
    Fetches the current weather for a given location (default: San Francisco).

    Args:
        location: The name of the city for which to retrieve the weather information.

    Returns:
        A dictionary containing weather information such as temperature in celsius, weather description, and humidity.
    """
    print(f"Getting current weather for {location}")
    apikey=WEATHER_API_KEY.get_secret_value()
    if apikey == "unset":
        print("No API key present; using a fixed, predetermined value for demonstration purposes")
        return {
            "description": "thunderstorms",
            "temperature": 25.3,
            "humidity": 94
        }

    try:
        # API request to fetch weather data
        weather_url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={apikey}&units=metric"
        weather_data = requests.get(weather_url)
        data = weather_data.json()
        # Extracting relevant weather details
        weather_description = data["weather"][0]["description"]
        temperature = data["main"]["temp"]
        humidity = data["main"]["humidity"]

        # Returning weather details
        return {
            "description": weather_description,
            "temperature": temperature,
            "humidity": humidity
        }
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return {
            "description": "none",
            "temperature": "none",
            "humidity": "none"
        }

print("=== Weather Tools Initialized ===")
print("  Tool 1: 'get_current_weather'")

## Step 5: Build the Route and Solve Agent

Now we'll build the Route and Solve agent. This agent consists of:
1. A **router node** that analyzes the query and determines which category of tools to use
2. Multiple **subagent nodes**, each specialized for a specific tool category
3. A **loop mechanism** that allows the router to receive feedback and reroute if needed

### Step 5.1: Define the agent state

In [ ]:
from typing import Annotated, TypedDict, Literal
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    # The subagent that was routed to
    current_subagent: str

print("State schema defined - Tracks messages and routing decisions for graph flow.")

### Step 5.2: Create subagent graphs

We create separate function calling agents for each tool category using the `create_agent` utility. Each subagent only sees tools relevant to its domain. The `create_agent` function encapsulates the manual graph building (nodes, edges, routing logic), providing a cleaner and more maintainable approach.

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import AIMessage

# Define finance tools and create finance subagent using create_agent utility
finance_tools = [get_stock_price]
finance_agent = create_agent(
    model=llm,
    tools=finance_tools,
)
print("Finance subagent created using create_agent - Bound with finance tools: get_stock_price")

# Define weather tools and create weather subagent using create_agent utility
weather_tools = [get_current_weather]
weather_agent = create_agent(
    model=llm,
    tools=weather_tools,
)
print("Weather subagent created using create_agent - Bound with weather tools: get_current_weather")

# Finance subagent node - wraps the created agent graph
def finance_llm_node(state: State) -> State:
    messages = state["messages"]
    # Use the finance_agent graph to get response
    input_state = {"messages": messages}
    # Execute the finance agent and get the final state
    final_state = finance_agent.invoke(input_state)
    response_message = final_state["messages"][-1]
    return State(messages=[response_message], current_subagent="finance")

print("Finance subagent node configured - Uses create_agent utility")

# Weather subagent node - wraps the created agent graph
def weather_llm_node(state: State) -> State:
    messages = state["messages"]
    # Use the weather_agent graph to get response
    input_state = {"messages": messages}
    # Execute the weather agent and get the final state
    final_state = weather_agent.invoke(input_state)
    response_message = final_state["messages"][-1]
    return State(messages=[response_message], current_subagent="weather")

print("Weather subagent node configured - Uses create_agent utility")

### Step 5.3: Create the router node

The router analyzes the user query and determines which subagent category should handle it.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

def router_node(state: State) -> State:
    """
    Router node that analyzes the query and routes to the appropriate subagent.
    
    Returns the state with a routing decision in the current_subagent field.
    The returned value should be 'finance', 'weather', or 'end' (for non-tool queries).
    """
    messages = state["messages"]
    user_query = messages[-1].content if messages else ""
    
    # Create a system prompt for the router
    router_system_prompt = """You are a router that directs queries to specialized agents.

Available agents:
- finance: Handles stock prices, market information, and financial queries
- weather: Handles weather forecasts, current weather, and climate information

Analyze the query and respond with ONLY the agent name (finance, weather, or none).
If the query doesn't clearly fit any agent, respond with 'none'."""
    
    # Call the model with proper system and user messages
    router_messages = [
        SystemMessage(content=router_system_prompt),
        HumanMessage(content=user_query)
    ]
    response = llm.invoke(router_messages)
    routing_decision = response.content.strip().lower()
    
    # Normalize the response
    if "finance" in routing_decision:
        route = "finance"
    elif "weather" in routing_decision:
        route = "weather"
    else:
        route = "none"
    
    print(f"[Router]: Routing decision - {route}")
    return State(messages=messages, current_subagent=route)

print("Router node defined - Analyzes queries and routes to appropriate subagent")

### Step 5.4: Define routing edges

These functions control the flow between nodes based on the router's decision and subagent responses.

In [ ]:
def route_to_subagent(state: State) -> str:
    """
    Routes to the appropriate subagent based on the router's decision.
    """
    subagent = state.get("current_subagent", "none")
    if subagent == "finance":
        return "finance_llm"
    elif subagent == "weather":
        return "weather_llm"
    else:
        return "end_node"

print("Conditional routing edges defined - Control flow logic for graph navigation")

### Step 5.5: Build the graph

Assemble all the nodes and edges into the complete Route and Solve agent graph.

In [ ]:
from langgraph.graph import StateGraph, START, END

# Create the graph
graph_builder = StateGraph(State)

# Add nodes
graph_builder.add_node("router", router_node)

# Finance subagent node
graph_builder.add_node("finance_llm", finance_llm_node)

# Weather subagent node
graph_builder.add_node("weather_llm", weather_llm_node)

# End node to format final response
def end_node(state: State) -> State:
    return state

graph_builder.add_node("end_node", end_node)

print("Step 1: All nodes added to graph - Router, subagents, and end node configured")

# Add edges
# Start -> Router (always start with routing)
graph_builder.add_edge(START, "router")

# Router -> Subagent routing
graph_builder.add_conditional_edges(
    "router",
    route_to_subagent,
    {
        "finance_llm": "finance_llm",
        "weather_llm": "weather_llm",
        "end_node": "end_node",
    }
)

# Subagents go directly to end node (create_agent handles tool calling internally)
graph_builder.add_edge("finance_llm", "end_node")
graph_builder.add_edge("weather_llm", "end_node")

# End node to END
graph_builder.add_edge("end_node", END)

print("Step 2: All edges configured - START -> Router -> Subagents -> END flow established")

# Compile the graph
graph = graph_builder.compile()

print("Step 3: Route and Solve Agent graph compiled successfully!")
print("\n=== Graph Structure ===")
print("- Router node: Directs queries to appropriate subagent")
print("- Finance subagent: Handles stock price queries using create_agent with get_stock_price")
print("- Weather subagent: Handles weather queries using create_agent with get_current_weather")
print("- Tool execution loops managed internally by create_agent subagents")

### Step 5.6: Visualize the graph

Let's visualize the structure of our Route and Solve agent graph.

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not display graph: {e}")
    print("This is optional and requires additional dependencies.")

## Step 6: Using the Route and Solve Agent

Now let's test the agent with various queries that will be routed to different subagents.

In [ ]:
from langchain_core.messages import HumanMessage

def route_and_solve_agent(graph, user_input: str):
    """
    Run the Route and Solve agent with a user query.
    
    The agent will:
    1. Route the query to the appropriate subagent
    2. Execute the subagent's tools as needed
    3. Return the final response
    """
    user_message = HumanMessage(user_input)
    print(f"\n{'='*60}")
    print(user_message.pretty_repr())
    print(f"{'='*60}\n")
    
    input_state = {"messages": [user_message], "current_subagent": ""}
    
    for event in graph.stream(input_state):
        for node_name, node_state in event.items():
            if "messages" in node_state and node_state["messages"]:
                print(f"[{node_name}] {node_state['messages'][-1].pretty_repr()}")
                print()

### Test 1: Finance Query

Test a query that should be routed to the finance subagent. The query should include both a ticker symbol and a date in YYYY-MM-DD format.

In [ ]:
route_and_solve_agent(graph, "What were the IBM stock prices on 2025-09-05?")

### Test 2: Weather Query

Test a query that should be routed to the weather subagent.

In [ ]:
route_and_solve_agent(graph, "What is the weather in San Francisco?")

### Test 3: General Knowledge Query

Test a query that doesn't require any specific tool category.

In [ ]:
route_and_solve_agent(graph, "What is the capital of France?")

### Future enhancements to this architecture:

- **Feedback loops**: Allow subagents to report back to the router if they can't find a tool they need
- **Tool chaining**: Enable subagents to request tools from other domains through the router
- **Confidence scoring**: Have the router return confidence levels along with routing decisions
- **Multi-tool queries**: Handle queries that require both finance and weather data by routing back through the router
